In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader, Dataset

In [2]:
from PIL import Image
import os

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

In [3]:
class WheatTest(Dataset):

    def __init__(self, image_ids, image_dir, transforms=None):
        super().__init__()
        self.image_ids = image_ids
        self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, idx: int):
        image_id = self.image_ids[idx]

        image = cv.imread(f'{self.image_dir}/{image_id}.jpg', cv.IMREAD_COLOR)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB).astype(np.float32)
#         image /= 255.0

        if self.transforms:
            sample = {
            'image': image,
            }
            sample = self.transforms(**sample)
            image = sample['image']

        return image, image_id

    def __len__(self) -> int:
        return len(self.image_ids)

    @staticmethod
    def get_test_transform():
        return A.Compose([
          ToTensorV2(p=1.0)
        ])


In [4]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# create a Faster R-CNN model without pre-trained
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False, pretrained_backbone=False)

num_classes = 7 # wheat or not(background)

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained model's head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# load the trained weights
model.load_state_dict(torch.load("/openbayes/input/input2/fasterrcnn_resnet50_fpn_6.pth", map_location=device))
model.eval()


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu

In [26]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import time

In [27]:
insect_classes=["Araneae","Coleoptera","Diptera","Hemiptera","Hymenoptera","Lepidoptera","Odonata"]

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
test_transform=A.Compose([
          ToTensorV2(p=1.0)
        ])


root="/openbayes/input/input0/"
# image = cv.imread(root+insect_classes[3]+"/000887d8cacc.jpg", cv.IMREAD_COLOR)
image = cv.imread("45.jpg", cv.IMREAD_COLOR)
img = cv.cvtColor(image, cv.COLOR_BGR2RGB).astype(np.float32)
img=img/255.0

In [28]:
sample = {
            'image': img,
    }
sample=test_transform(**sample)
img=sample["image"]

In [29]:
start=time.time()
output = model([img])
end=time.time()
print(start-end)
print(output)

-27.574568033218384
[{'boxes': tensor([[114.0001, 104.6395, 417.4249, 444.6251],
        [117.3470,  65.0695, 416.7137, 421.9429]], grad_fn=<StackBackward>), 'labels': tensor([3, 1]), 'scores': tensor([0.8070, 0.5451], grad_fn=<IndexBackward>)}]


In [32]:
# output=output[0]
boxes=output["boxes"].tolist()
labels=output["labels"].tolist()
scores=output["scores"].tolist()

print(boxes)
print(labels)
print(scores)
for i in range(len(scores)):
    if scores[i]>0.5:
        box=boxes[i]
        cv.rectangle(image, (int(box[0]),int(box[1])), (int(box[2]),int(box[3])), (0,255,0), 2)
                         
cv.imwrite("r1.png",image)

[[114.0001449584961, 104.63947296142578, 417.4248962402344, 444.6250915527344], [117.34700012207031, 65.06951904296875, 416.7137451171875, 421.94293212890625]]
[3, 1]
[0.8070316314697266, 0.5450823903083801]


True